In [1]:
import sys;
from esinet.evaluate import eval_mse
# sys.path.insert(0, '../')
from esinet import Simulation, Net, util, evaluate
from forward import create_forward_model, get_info, create_forward_model_gai
from matplotlib import pyplot as plt
from esinet.util import calculate_source
import numpy as np
import mne

In [2]:
# Create generic Forward Model 设置频率
info = get_info(sfreq=256)

# 添加电极扰动
# pos = np.array([info['chs'][i]['loc'][:3] for i in range(len(info['chs']))]) # 获取电极坐标
# print(pos)
# random_perturbation = np.random.normal(0, 0.005, pos.shape) # 为每个坐标添加高斯分布的随机扰动
# perturbed_pos = pos + random_perturbation # 添加随机扰动
# for i, ch in enumerate(info['chs']):
#     ch['loc'][:3] = perturbed_pos[i]
# pos2 = np.array([info['chs'][i]['loc'][:3] for i in range(len(info['chs']))]) # 获取电极坐标
# print(pos2)

# 生成前向模型，ico2表示324个源，ico3表示1284个源，oct5表示2052个源,其间距为10mm ico4表示5124个源,
#fwd = create_forward_model(info=info, sampling='ico3')
fwd = create_forward_model_gai(info=info, sampling='ico3') # 为了生成不同导电率的前向模型
leadfield, pos = util.unpack_fwd(fwd)[1:3]
print(fwd)
#fwd.save('fwd_daodianlv.fif', overwrite=True)

(0.3, 0.012, 0.3)
Creating the BEM geometry...
Going from 4th to 4th subdivision of an icosahedron (n_tri: 5120 -> 5120)
Going from 4th to 4th subdivision of an icosahedron (n_tri: 5120 -> 5120)
Going from 4th to 4th subdivision of an icosahedron (n_tri: 5120 -> 5120)
outer skin  CM is  -0.12  -0.14  22.79 mm
outer skull CM is   0.70  -9.88  44.01 mm
inner skull CM is   0.67 -10.01  44.26 mm
Checking that surface outer skull is inside surface outer skin  ...
Checking that surface inner skull is inside surface outer skull ...
Checking distance between outer skin  and outer skull surfaces...
Minimum distance between the outer skin  and outer skull surfaces is approximately    3.7 mm
Checking distance between outer skull and inner skull surfaces...
Minimum distance between the outer skull and inner skull surfaces is approximately    3.5 mm
Surfaces passed the basic topology checks.
Complete.

Three-layer model surfaces loaded.
Computing the linear collocation solution...
    Matrix coeffi

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


    Found  32/375 points outside using solid angles
    Total 472/642 points inside the surface
Interior check completed in 3248.8 ms
    170 source space points omitted because they are outside the inner skull surface.


[Parallel(n_jobs=-1)]: Done   4 out of  12 | elapsed:    3.0s remaining:    6.2s
[Parallel(n_jobs=-1)]: Done   7 out of  12 | elapsed:    3.1s remaining:    2.2s
[Parallel(n_jobs=-1)]: Done  10 out of  12 | elapsed:    3.1s remaining:    0.5s
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:    3.1s finished


    56 source space points omitted because of the    5.0-mm distance limit.
Checking surface interior status for 642 points...
    Found 141/642 points inside  an interior sphere of radius   43.6 mm
    Found  25/642 points outside an exterior sphere of radius   91.8 mm
    Found 113/476 points outside using surface Qhull
    Found  30/363 points outside using solid angles
    Total 474/642 points inside the surface
Interior check completed in 130.2 ms
    168 source space points omitted because they are outside the inner skull surface.
    46 source space points omitted because of the    5.0-mm distance limit.

Setting up for EEG...


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of  12 | elapsed:    0.0s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done   7 out of  12 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  10 out of  12 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:    0.0s finished


Computing EEG at 844 source locations (free orientations)...


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.



Finished.
<Forward | MEG channels: 0 | EEG channels: 60 | Source space: Surface with 844 vertices | Source orientation: Fixed>


[Parallel(n_jobs=-1)]: Done   4 out of  12 | elapsed:    1.5s remaining:    3.1s
[Parallel(n_jobs=-1)]: Done   7 out of  12 | elapsed:    1.5s remaining:    1.1s
[Parallel(n_jobs=-1)]: Done  10 out of  12 | elapsed:    1.5s remaining:    0.2s
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:    1.5s finished


In [3]:
settings = {
    'method': 'standard', # 模拟源信号时间的方式
    'number_of_sources': (1,10), # 源的数量范围
    'extents':  (15,35),  # 源的直径范围 单位是mm
    'amplitudes': (3, 10), # 源的单流大小范围，单位是nAm，前面的n表示纳，是10的-9次方，1 nAm = 0.001 µAm，默认(0.001, 100)
    'shapes': 'mixed', # 源的形状，可以是高斯分布或者均匀分布，以及二者混合分布
    'duration_of_trial': 0.1, #试次的时间长度，以秒为单位
    'sample_frequency': 256, # 数据的采样频率
    'target_snr': (5, 20), # 目标信噪比范围，默认（2，20）
    'beta': (0.5, 1.5), 
    'source_spread': "region_growing", # 源的空间分布
    'source_number_weighting': True, # 指示是否对源的数量进行加权，也就是从设定的范围内随机选择一个值作为源的数量
    'source_time_course': "random", # 源的时间过程类型是随机
}
sim = Simulation(fwd, info, settings=settings, verbose=True)
sim.simulate(n_samples=1000)

D:\jupyter_note\SWX_source\Simulated_data\esinet\simulation.py:701: RuntimeWarning: 34.3% of original source space vertices have been omitted, tri-based adjacency will have holes.
Consider using distance-based adjacency or morphing data to all source space vertices.
  adj = mne.spatial_src_adjacency(self.fwd["src"], verbose=False).toarray().astype(int)#使用MNE工具函数`mne.spatial_src_adjacency`计算前向模型的源空间(`self.fwd["src"]`)的邻接矩阵(`adj`)，此矩阵表示源之间的邻接关系


Simulate Source
Simulating data based on sparse patches.


100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:03<00:00, 311.75it/s]


Converting Source Data to mne.SourceEstimate object


100%|███████████████████████████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 21541.95it/s]


...

1 matching events found
No baseline correction applied
0 projection items activated
Not setting metadata
1 matching events found
No baseline correction applied
0 projection items activated
Not setting metadata
1 matching events found
No baseline correction applied
0 projection items activated
Not setting metadata
1 matching events found
No baseline correction applied
0 projection items activated
Not setting metadata
1 matching events found
No baseline correction applied
0 projection items activated
Not setting metadata
1 matching events found
No baseline correction applied
0 projection items activated
Not setting metadata
1 matching events found
No baseline correction applied
0 projection items activated
Not setting metadata
1 matching events found
No baseline correction applied
0 projection items activated
Not setting metadata
1 matching events found
No baseline correction applied
0 projection items activated
Not setting metadata
1 matching events found
No baseline correction applied


No baseline correction applied
0 projection items activated
Not setting metadata
1 matching events found
No baseline correction applied
0 projection items activated
Not setting metadata
1 matching events found
No baseline correction applied
0 projection items activated
Not setting metadata
1 matching events found
No baseline correction applied
0 projection items activated
Not setting metadata
1 matching events found
No baseline correction applied
0 projection items activated
Not setting metadata
1 matching events found
No baseline correction applied
0 projection items activated
Not setting metadata
1 matching events found
No baseline correction applied
0 projection items activated
Not setting metadata
1 matching events found
No baseline correction applied
0 projection items activated
Not setting metadata
1 matching events found
No baseline correction applied
0 projection items activated
Not setting metadata
1 matching events found
No baseline correction applied
0 projection items activ

0 projection items activated
Not setting metadata
1 matching events found
No baseline correction applied
0 projection items activated
Not setting metadata
1 matching events found
No baseline correction applied
0 projection items activated
Not setting metadata
1 matching events found
No baseline correction applied
0 projection items activated
Not setting metadata
1 matching events found
No baseline correction applied
0 projection items activated
Not setting metadata
1 matching events found
No baseline correction applied
0 projection items activated
Not setting metadata
1 matching events found
No baseline correction applied
0 projection items activated
Not setting metadata
1 matching events found
No baseline correction applied
0 projection items activated
Not setting metadata
1 matching events found
No baseline correction applied
0 projection items activated
Not setting metadata
1 matching events found
No baseline correction applied
0 projection items activated
Not setting metadata
1 mat

Not setting metadata
1 matching events found
No baseline correction applied
0 projection items activated
Not setting metadata
1 matching events found
No baseline correction applied
0 projection items activated
Not setting metadata
1 matching events found
No baseline correction applied
0 projection items activated
Not setting metadata
1 matching events found
No baseline correction applied
0 projection items activated
Not setting metadata
1 matching events found
No baseline correction applied
0 projection items activated
Not setting metadata
1 matching events found
No baseline correction applied
0 projection items activated
Not setting metadata
1 matching events found
No baseline correction applied
0 projection items activated
Not setting metadata
1 matching events found
No baseline correction applied
0 projection items activated
Not setting metadata
1 matching events found
No baseline correction applied
0 projection items activated
Not setting metadata
1 matching events found
No baselin

1 matching events found
No baseline correction applied
0 projection items activated
Not setting metadata
1 matching events found
No baseline correction applied
0 projection items activated
Not setting metadata
1 matching events found
No baseline correction applied
0 projection items activated
Not setting metadata
1 matching events found
No baseline correction applied
0 projection items activated
Not setting metadata
1 matching events found
No baseline correction applied
0 projection items activated
Not setting metadata
1 matching events found
No baseline correction applied
0 projection items activated
Not setting metadata
1 matching events found
No baseline correction applied
0 projection items activated
Not setting metadata
1 matching events found
No baseline correction applied
0 projection items activated
Not setting metadata
1 matching events found
No baseline correction applied
0 projection items activated
Not setting metadata
1 matching events found
No baseline correction applied


In [4]:
# 提取
def prep_data(sim):
    
    x = np.squeeze(np.stack([eeg.average().data.astype(np.float32) for eeg in sim.eeg_data]))
    y = np.squeeze(np.stack([src.data.astype(np.float32) for src in sim.source_data]))
    
    for i, (x_sample, y_sample) in enumerate(zip(x, y)):
        # X[i] = np.stack([(x - np.mean(x)) / np.std(x) for x in x_sample.T], axis=0).T
        # y[i] = np.stack([ y / np.max(abs(y)) for y in y_sample.T], axis=0).T

        x[i] = np.stack([x - np.mean(x) for x in x_sample.T], axis=0).T
        x[i] /= x[i].std()
        y[i] /= np.max(abs(y[i]))
        
    x = np.swapaxes(x, 1,2)
    y = np.swapaxes(y, 1,2)
    
    return X, y

In [5]:
y = np.squeeze(np.stack([src.data.astype(np.float32) for src in sim.source_data]))
print(y.shape)

(1000, 844, 26)


In [6]:
# 提取源信号
sources = sim.source_data
sources = [source.data for source in sources]
print(type(sources))
print(type(sources[0]))

<class 'list'>
<class 'numpy.ndarray'>


In [7]:
y=np.array(sources)
print(type(y))
print(y.shape)

<class 'numpy.ndarray'>
(1000, 844, 26)


In [8]:
x=np.array(sim.eeg_data)
print(type(x))
print(x.shape)
x=x.squeeze()
print(x.shape)

<class 'numpy.ndarray'>
(1000, 1, 60, 26)
(1000, 60, 26)


In [9]:
np.save('x_test_daodianlv.npy', x)

In [10]:
np.save('y_test_daodianlv.npy', y)

In [11]:
#np.save('pos_daodianlv.npy', pos)

In [12]:
import pickle

# 指定要保存到的文件名
file_name = 'sim_daodianlv.pkl'

# 打开文件，使用二进制写模式
with open(file_name, 'wb') as f:
    # 使用pickle.dump()函数将对象保存到文件中
    pickle.dump(sim, f)